```
        
        |   | /  |   |  |  |  |  |  |  /  |   |  |  |  |      [0][1][0]   
        |       /    |  |  |  |  |       /    \   \_/  /      [1][1][1]   
        |      /     |  |  |  |  |      /      \  \_/ /       [1][1][1]   
        |  |\  \     |  '--'  |  |  |\  \       \    /        Happy Hacking    
        | _| `.__\   |________|  | _| `.__\      |___|                                
```

*Author: Christian Camilo Urcuqui López*

*Date: 04 November 2019*

*GitHub: https://github.com/urcuqui/ *

**Content**

+ [Android Malware Analysis ](#Android-Malware-Analysis)
    + [Android](#Android)
+ [State of the Art](#State-of-the-Art)
+ [Datasets](#Datasets)
+ [Static Analysis](#Static-Analysis)
    + [Exploratory](#Exploratory)
    + [Modeling](#Modeling)
+ [Dynamic Analysis](#Dynamic-Analysis)
    + [Data Cleaning and Processing](#Data-Cleaning-and-Processing)
    + [Modeling](#Modeling)
+ [Conclusions](#Conclusions)
+ [References](#References)

In [1]:
from datetime import datetime

print("last update: {}".format(datetime.now())) 

# Android Malware Analysis

In this notebook I would like to share some of my research results, I'm going to explain to the interested of this cybersecurity area how I used data science in order to train machine learning mechanisms to detect threats, especially, for this part, I abstracted some of the ideas from my book [1] and others articles, which I will use during the explanation about this topic. 

This research is for me as my philanthropy work, so please the idea of this sharing is to get feedback, networking and cites about these results (hacker culture ☠)

<img src="https://pbs.twimg.com/media/Dw_dCMfX0AAwLUf.jpg" height="250" width="250"/>

## Android


Android is one of the most used mobile operating systems worldwide. Due to its technological impact, its open-source code and the possibility of installing applications from third parties without any central control, Android has recently become a malware target. Even if it includes security mechanisms, the last news about malicious activities and Android´s vulnerabilities point to the importance of continuing the development of methods and frameworks to improve its security.

To prevent malware attacks, researches and developers have proposed different security solutions, applying static analysis, dynamic analysis, and artificial intelligence. Indeed, data science has become a promising area in cybersecurity, since analytical models based on data allow for the discovery of insights that can help to predict malicious activities.

We can analyze cyber threats using two techniques, static analysis, and dynamic analysis, the most important thing is that these are the approaches to get the features that we are going to use in data science.

+ **Static analysis**: it includes the methods that allow us to get information about the software that we want to analyze without executing it, one example of them is the study of the code, their callings, resources, etc.
+ **Dynamic analysis**: it is another approach where the idea is to analyze the cyber threat during its execution, in other words, get information about its behavior, some of their features are the netflows.


# State of the Art

In 2016 we published an article [2] about the state of the art of frameworks and results about Android malware detection. This work reflects different static analysis tools (TaintDroid, Stowaway, Crowdroid y Airmid), dynamic analysis systems (Paranoid and DroidMOSS), frameworks (MobSafe, SAAF, and ASEF) and some research results about using machine learning. From this article we concluded that the idea is using both static and dynamic analysis in order to get spectra of features, moreover, some works have been working to use virtual devices in the cloud.


# Datasets

In 2016 we explored [3] Android Genome Project (MalGenome), it is a dataset which was active from 2012 until the end of the year 2015, this set of malware has a size of 1260 applications, grouped into a total of 49 families. Today, we can find other jobs such as: Drebin, a research project offering a total of 5560 applications consisting of 179 malware families; AndrooZoo, which includes a collection of 5669661 applications Android from different sources (including Google Play); VirusShare, another repository that provides samples of malware for cybersecurity researchers; and DroidCollector, this is another set which provides around 8000 benign applications and 5560 malware samples, moreover, it facilitates us samples of network traffic as pcap files.

# Static Analysis 

In this first step, I'm going to analyze some features in order to answer the next hypothesis, *exist a differential of the permissions used between a set of malware and benign samples*, in other words… 

<img src="https://pics.me.me/when-the-flashlight-app-wants-access-to-your-call-history-32812256.png" height="250" width="250">

For this approach, I developed a code that consisted to extract and make a CSV file which has information about permissions of applications, through this script you can map each APK (Android Application Package) against a list of permissions. You can find more information about the proposed framework at [3]

https://github.com/urcuqui/WhiteHat

### Packages

In [1]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

from sklearn import preprocessing
import torch
from sklearn import svm
from sklearn import tree
import pandas as pd
from sklearn.externals import joblib
import pickle
import numpy as np
import seaborn as sns

### Exploratory 

For the next analysis, I'm going to explore the Malgenome dataset, as I said nowadays we can find other sources with a lot of examples and malware families which would be important for future works, the idea of the next experiment and results is to show our first approached.

<img src="https://ieee-dataport.org/sites/default/files/styles/large/public/Sin%20t%C3%ADtulo.png?itok=4YoQim00" /> 

*It is not neccesary to apply a data cleaning and transformation process*

In [1]:
import pandas as pd
df = pd.read_csv("../input/datasetandroidpermissions/train.csv", sep=";")

In [1]:
df = df.astype("int64")
df.type.value_counts()

Type is the label that represents if an application is a malware or not, as we can see this dataset is balanced.

In [1]:
df.shape

*Let's get the top 10 of permissions that are used for our malware samples*

*Malicious*

In [1]:
pd.Series.sort_values(df[df.type==1].sum(axis=0), ascending=False)[1:11]

*Benign*

In [1]:
pd.Series.sort_values(df[df.type==0].sum(axis=0), ascending=False)[:10]

In [1]:
import matplotlib.pyplot as plt
fig, axs =  plt.subplots(nrows=2, sharex=True)

pd.Series.sort_values(df[df.type==0].sum(axis=0), ascending=False)[:10].plot.bar(ax=axs[0])
pd.Series.sort_values(df[df.type==1].sum(axis=0), ascending=False)[1:11].plot.bar(ax=axs[1], color="red")

The last outputs allow us to get insights about a difference between the permissions used by the malware and the benign applications.

### Modeling

In [1]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:330], df['type'], test_size=0.20, random_state=42)

*Naive Bayes algorithm*

In [1]:
# Naive Bayes algorithm
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# pred
pred = gnb.predict(X_test)

# accuracy
accuracy = accuracy_score(pred, y_test)
print("naive_bayes")
print(accuracy)
print(classification_report(pred, y_test, labels=None))

*kneighbors algorithm*

In [1]:
# kneighbors algorithm

for i in range(3,15,3):
    
    neigh = KNeighborsClassifier(n_neighbors=i)
    neigh.fit(X_train, y_train)
    pred = neigh.predict(X_test)
    # accuracy
    accuracy = accuracy_score(pred, y_test)
    print("kneighbors {}".format(i))
    print(accuracy)
    print(classification_report(pred, y_test, labels=None))
    print("")

*Decision Tree*

In [1]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Read the csv test file

pred = clf.predict(X_test)
# accuracy
accuracy = accuracy_score(pred, y_test)
print(clf)
print(accuracy)
print(classification_report(pred, y_test, labels=None))

Through the last results we can see how we trained different classifiers to detect malware using its permissions, but as I said this is only a first approximation, I didn't analyze the hyperparameters and others things to improve the results.

# Dynamic Analysis

For this approach, we used a set of pcap files from the DroidCollector project integrated by 4705 benign and 7846 malicious applications. All of the files were processed by our feature extractor script (a result from [4]), the idea of this analysis is to answer the next question, according to the static analysis previously seen a lot of applications use a network connection, in other words, they are trying to communicate or transmit information, so.. is it possible to distinguish between malware and benign application using network traffic?

In [1]:
import pandas as pd
data = pd.read_csv("../input/network-traffic-android-malware/android_traffic.csv", sep=";")
data.head()

In [1]:
data.columns

In [1]:
data.shape

In [1]:
data.type.value_counts()

In this case, we have an unbalanced dataset, so another model evaluation will be used.

### Data Cleaning and Processing

In [1]:
data.isna().sum()

When we processed each pcap we had some problems getting three features (duration, avg remote package rate, avg local package rate) this why got during the feature processing script, we don't have this issue nowadays. 

In [1]:
data = data.drop(['duracion','avg_local_pkt_rate','avg_remote_pkt_rate'], axis=1).copy()

In [1]:
data.describe()

Now, the idea is to see the outliers in the data

In [1]:
sns.boxplot(data.tcp_urg_packet)

In [1]:
data.loc[data.tcp_urg_packet > 0].shape[0]

That column will be no used for the analysis, only two rows are different to zero, maybe they are interesting for future analysis.

In [1]:
data = data.drop(columns=["tcp_urg_packet"], axis=1).copy()
data.shape

In [1]:
sns.pairplot(data)

We have many outliers in some features, I will omit the depth analysis and only get the set of the data without the noise. 

In [1]:
data=data[data.tcp_packets<20000].copy()
data=data[data.dist_port_tcp<1400].copy()
data=data[data.external_ips<35].copy()
data=data[data.vulume_bytes<2000000].copy()
data=data[data.udp_packets<40].copy()
data=data[data.remote_app_packets<15000].copy()

In [1]:
data[data.duplicated()].sum()

In [1]:
data=data.drop('source_app_packets.1',axis=1).copy()

In [1]:
scaler = preprocessing.RobustScaler()
scaledData = scaler.fit_transform(data.iloc[:,1:11])
scaledData = pd.DataFrame(scaledData, columns=['tcp_packets','dist_port_tcp','external_ips','vulume_bytes','udp_packets','source_app_packets','remote_app_packets',' source_app_bytes','remote_app_bytes','dns_query_times'])

From [6] we concluded that the best network features are:

+ (R1): TCP packets, it has the number of packets TCP sent and got during communication.
+ (R2): Different TCP packets, it is the total number of packets different from TCP.
+ (R3): External IP, represents the number the external addresses (IPs) where the application tried to communicated
+ (R4): Volume of bytes, it is the number of bytes that was sent from the application to the external sites
+ (R5) UDP packets, the total number of packets UDP transmitted in a communication.
+ (R6) Packets of the source application, it is the number of packets that were sent from the application to a remote server.
+ (R7) Remote application packages, number of packages received from external sources.
+ (R8) Bytes of the application source, this is the volume (in Bytes) of the communication between the application and server.
+ (R9) Bytes of the application remote, this is the volume (in Bytes) of the data from the server to the emulator.
+ (R10) DNS queries, number of DNS queries.


### Modeling

In [1]:
X_train, X_test, y_train, y_test = train_test_split(scaledData.iloc[:,0:10], data.type.astype("str"), test_size=0.25, random_state=45)

In [1]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
pred = gnb.predict(X_test)
## accuracy
accuracy = accuracy_score(y_test,pred)
print("naive_bayes")
print(accuracy)
print(classification_report(y_test,pred, labels=None))
print("cohen kappa score")
print(cohen_kappa_score(y_test, pred))

In [1]:
# kneighbors algorithm

for i in range(3,15,3):
    
    neigh = KNeighborsClassifier(n_neighbors=i)
    neigh.fit(X_train, y_train)
    pred = neigh.predict(X_test)
    # accuracy
    accuracy = accuracy_score(pred, y_test)
    print("kneighbors {}".format(i))
    print(accuracy)
    print(classification_report(pred, y_test, labels=None))
    print("cohen kappa score")
    print(cohen_kappa_score(y_test, pred))
    print("")

In [1]:
rdF=RandomForestClassifier(n_estimators=250, max_depth=50,random_state=45)
rdF.fit(X_train,y_train)
pred=rdF.predict(X_test)
cm=confusion_matrix(y_test, pred)

accuracy = accuracy_score(y_test,pred)
print(rdF)
print(accuracy)
print(classification_report(y_test,pred, labels=None))
print("cohen kappa score")
print(cohen_kappa_score(y_test, pred))
print(cm)

# Conclusions

**Great 💪🤖!!**, now we have seen the two approaches to analyze a cyber threat. Of course we can use a lot of variables that in this case we didn't use them, for example netflows, methods callings, graph analysis, and many others, but the idea behind this work is to understand that we need to pay attention of all of the environments because when we are working in **cybersecurity** we face with a **complex problem**. 

I'm going to continue with my cybersecurity kernels, in spite of this is the first one, this kernel will be updated once I find an improvement and new results.

Some of the next kernels that I will publish are:
+ Malicious Websites Detection
+ Secure Learning
+ Cyber Web Attacks Detection
+ Deep Fakes Detection

I would like to mention my ex-bachelor students who helped me to continue in my Android research
+ Melisa Garcia
+ Jose Luis Osorio
+ Andres Felipe Perez
+ Jhoan Steven Delgado

Morevoer, I would like to cite the co-authors and my english teacher:

+ Andres Navarro 
+ Sebastian Londoño
+ Samir Riascos

# References

+ [1] López, U., Camilo, C., García Peña, M., Osorio Quintero, J. L., & Navarro Cadavid, A. (2018). Ciberseguridad: un enfoque desde la ciencia de datos-Primera edición.
+ [2] Navarro Cadavid, A., Londoño, S., Urcuqui López, C. C., & Gomez, J. (2014, June). Análisis y caracterización de frameworks para detección de aplicaciones maliciosas en Android. In Conference: XIV Jornada Internacional de Seguridad Informática ACIS-2014 (Vol. 14). ResearchGate.
+ [3] Urcuqui-López, C., & Cadavid, A. N. (2016). Framework for malware analysis in Android.
+ [4] Urcuqui,  C.,  Navarro,  A.,  Osorio,  J.,  &  Garcıa,  M.  (2017). Machine Learning  Classifiers  to  Detect  Malicious  Websites. CEUR  Workshop Proceedings. Vol 1950, 14-17.
+ [5] López, C. C. U., Villarreal, J. S. D., Belalcazar, A. F. P., Cadavid, A. N., & Cely, J. G. D. (2018, May). Features to Detect Android Malware. In 2018 IEEE Colombian Conference on Communications and Computing (COLCOM) (pp. 1-6). IEEE.